# Retail Pricing Information Distribution Prototype

### Paul M. Washburn

This is a first attempt at developing a prototype to automate the process of reporting current and future pricing to retail accounts.  While the overall process is similar across retailers, each one has a unique set of preferences, file types and information that they supply and demand.  

QuikTrip is the first we will inquire with.  

The goal is to build object-oriented data structures that enable easy processing of inputs and easy generation of outputs in order to save thousands of man-hours that are currently dedicated to this task, thus freeing them from playing defense to playing offense for the firm.  

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
from datetime import date
from collections import OrderedDict
import warnings
from matplotlib import pyplot as plt
from pandas.tseries.offsets import YearEnd
from pandas.tseries.holiday import USFederalHolidayCalendar
import seaborn as sns

warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 1000)

print('''
Notes to Developer:

This process is designed to be extensible, and thus it uses object oriented design in the
Python language.  This design is to ensure extensibility (i.e. adaptation to new and changing
demands) and memory/computational efficiency.  

All objects are stored in dictionaries for key-value access and to eliminate redundancy of 
what should be unique data objects.  
''')

# `Product` Objects to Represent Product Elements

The `Product` is the most important object in this process.  The logic calls for many product attributes, then some that it inherits from the (yet-to-be-defined) `ProductPricing` class.  `ProductPricing` will be implemented from the AS400's Frontline report.  

## Read in Raw Data from PRD1 in AS400

The query `pw_prd1` queries the AS400 for all the data that is needed in this process.  If need be, it can be extended to fit changing circumstances.  `Product` object instances are created using this data, and thus it must be refreshed every time this report is run.  

In [ ]:
def fetch_pw_prd1(path):
    '''
    Fetches product attributes for merging in later
    '''
    df_prd = pd.read_csv(path + 'pw_prd1.csv', encoding='latin1')
    
    # rename columns
    new_cols = {'PPROD#': 'product_id', 'PDESC': 'description', 'PSIZE@': 'size_code', 'PCLAS@': 'class_code', 
                'PQTYPC': 'qpc', 'PSUPPL': 'supplier_id', 'PBRAN#': 'brand_id', 'PWHSEU': 'warehouse_usage_code', 
                'UPC122': 'upc_code_12_2', 'PUPC': 'upc_code', 'PUPC12': 'upc_code_12_1', 'PUPCS3': 'upc_code_subfield',
                'PSQTYS': 'smallest_qty_sold', 'PUPC2': 'upc_code_2', 'UPC123': 'upc_code_12_3', 'PTYPE': 'type_code', 
                'PSIZE': 'size', 'PSIZE#': 'size_code_redef',  'PPROF': 'proof','PWINTP': 'wine_type', 
                'PCOSEL': 'co_sell_codes', 'PFOB': 'fob_cost_case', 'PPALSZ': 'pallet_size', 'PTIE': 'ties',
                'PTIER': 'tiers', 'PBPBKC': 'price_book_code', 'PBDESC': 'price_book_description', 'PVINTG': 'vintage',
                'PCOLR': 'color', 'PGIFT': 'gift_set', 'PCOPAK': 'co_pack'}
    df_prd.rename(columns=new_cols, inplace=True)
    
    return df_prd

DATASETS = {}

# read in products
df_all_products = fetch_pw_prd1(path='C:\\Users\\pmwash\\Desktop\\Re-Engineered Reports\\Generalized Lookup Data\\')
DATASETS['products_raw'] = df_all_products

print(DATASETS['products_raw'].head())

## Creating the `Product` Object

The `Product` object is one of the main objects in any solution architecture to this problem.  Certain base attributes and methods must be instantiated before more complex inheritances may occur.  

In [ ]:
class Product(object):
    def __init__(self, product_id, description, size_code, qpc, supplier_id, 
                 brand_id, class_code, type_code, upc_permutations,
                 warehouses_carried, co_sell_codes, price_book_code, private_label,
                 seasonality, cont_type, pallet_dims, proof, vintage, color, gift_set):
        '''
        Product represents a unique Product in our portfolio. All data needed can be obtained 
        by running the AS400 query pw_prd1.
        
        Instantiate with:
        --------------------------------------------------
         product_id: internal product id
         description: string describing product
         size_code: size code
         qpc: quantity per case
         supplier_id: will be replaced with object in future
         brand_id: brand identifier
         class_code: class code
         type_code: type code
         upc_permutations: dict of all UPC permutations
         warehouses_carried: list of warehouses carried
         co_sell_codes: string of 0's and 1's indicating which company sells
         price_book_code: boolean
         private_label: boolean
         seasonality: {'seas_code': val, 'seas_desc': val}
         cont_type: string describing retail container
         pallet_dims: {'pallet_size': val, ties': val, 'tiers': val}
         proof: alcoholic content
         vintage: wine vintage
         color: product color
         gift_set: boolean True if gift set, o.w. False
        '''
        self.product_id = product_id
        self.description = description
        self.size_code = size_code
        self.qpc = qpc
        self.supplier_id = supplier_id
        self.brand_id = brand_id
        self.class_code = class_code
        self.type_code = type_code
        self.upc_permutations = upc_permutations
        self.warehouses_carried = warehouses_carried
        self.co_sell_codes = {'kc': str(co_sell_codes)[0]=='1',
                              'stl': str(co_sell_codes)[1]=='1',
                              'col': str(co_sell_codes)[2]=='1',
                              'cape': str(co_sell_codes)[3]=='1',
                              'spfd': str(co_sell_codes)[4]=='1'}
        self.price_book_code = price_book_code
        self.private_label = private_label
        self.seasonality = seasonality
        self.cont_type = cont_type
        self.pallet_dims = pallet_dims
        
    def __str__(self):
        return '''
        Product object:
        ----------------------------------------------------------------
        Product:                    %s  -  %s
        Supplier:                   %s
        Company Sell Codes          %s
        ''' %(self.product_id, self.description, 
              self.supplier_id,
              self.co_sell_codes)
        
    def make_private_label(self):
        self.private_label = True
        
    def append_upcs(self, new_upc_dict):
        self.upc_permutations.update(new_upc_dict)

## Instantiate `Product` Objects and Save to `PRODUCT_MASTER` `dict` Object for Use

A base object is created for each `Product` in order to have their attributes flow down to the `ProductPricing` class which is specified below.  

In [ ]:
# get columns for later
upc_columns = [col for col in DATASETS['products_raw'].columns if 'upc' in col]
pallet_columns = ['pallet_size', 'ties', 'tiers']

# instantiate Product objects in list
PRODUCT_MASTER = {}
for i, product_id in enumerate(DATASETS['products_raw']['product_id'].tolist()): #already unique bc from PRD1
    # extract all needed dicts from data
    upc_permutations = DATASETS['products_raw'].loc[i, upc_columns].to_dict()
    pallet_dims = DATASETS['products_raw'].loc[i, pallet_columns].to_dict()
    
    # derive seasonality
    seasonality = {} #placeholder -- WHERE TO GET?  may need a setter method to set from frontline
    
    # cont_type same as above
    cont_type = None
    
    # mark private labels placeholder - will be updated below
    private_label = False
    
    # get companies sold at 
    warehouses_carried = []
    
    # actually instantiate product and save to PRODUCT_MASTER
    PRODUCT_MASTER[product_id] = Product(product_id, 
                                         DATASETS['products_raw'].loc[i, 'description'],
                                         DATASETS['products_raw'].loc[i, 'size_code'],
                                         DATASETS['products_raw'].loc[i, 'qpc'], 
                                         DATASETS['products_raw'].loc[i, 'supplier_id'],
                                         DATASETS['products_raw'].loc[i, 'brand_id'],
                                         DATASETS['products_raw'].loc[i, 'class_code'],
                                         DATASETS['products_raw'].loc[i, 'type_code'],
                                         upc_permutations,
                                         warehouses_carried, #ask holly how to parse
                                         DATASETS['products_raw'].loc[i, 'co_sell_codes'],
                                         DATASETS['products_raw'].loc[i, 'price_book_code'],
                                         private_label,
                                         seasonality,
                                         cont_type,
                                         pallet_dims,
                                         DATASETS['products_raw'].loc[i, 'proof'],
                                         DATASETS['products_raw'].loc[i, 'vintage'],
                                         DATASETS['products_raw'].loc[i, 'color'],
                                         DATASETS['products_raw'].loc[i, 'gift_set'])

In [ ]:
# print face of product obj
print(PRODUCT_MASTER[31125])

## `ProductPricing` Class to Represent Frontline Pricing at a Given Point in Time

The AS400 Frontline report is used as input for instantiating objects of this class.  

### Read in Frontline Data

The `AS400FrontLineReport` is not a central object, yet it is important that it exists.  By creating an object that has well-defined features we can better plan for the future ERP change, wherein this object will be re-configured and re-named (and eventually sunsetted) with updated logic.  

In [ ]:
as400_date = lambda dat: dt.strptime(str(dat)[-6:], '%y%m%d')

class AS400FrontLineReport(object):
    def __init__(self, path):
        self.data = self.read_and_process_frontline(path)
        self.run_date = dt.now()
        
    def __getattr__(self, attr):
        if hasattr(self, attr):
            return self.attr

    def read_and_process_frontline(self, path):
        # read in frontline query
        df = pd.read_csv(path + 'FRONTLNST.csv', header=0, encoding='latin-1')

        # replace blank space in column names
        df.columns = [str(col).replace(' ', '_') for col in df.columns]
        df.columns = [str(col).replace('#', 'ID') for col in df.columns]
        df.columns = [str(col).lower() for col in df.columns]

        # process specific columns
        df['effective_date'] = df['effective_date'].apply(as400_date)
        
        # rename columns to match
        dis = ['upcinner', 'upcinner_type', 'upccase', 'upccase_type', 'upcbshort', 'upcishort', 'upccshort', 'upcbttl', 'upcbttl_type']
        dat = ['upc_inner', 'upc_inner_type', 'upc_case', 'upc_case_type', 'upc_b_short', 'upc_i_short', 'upc_c_short', 'upc_btl', 'upc_btl_type']
        new_names = dict(zip(dis, dat))
        df.rename(columns=new_names, inplace=True)
        
        dis = ['sell_prc_1', 'nm_sell_prc_1', 'nm_+_1_sell_prc_1', 'nm_+_2_sell_prc_1', 'sell_prc_2', 'nm_sell_prc_2', 'nm_+_1_sell_prc_2', 'nm_+_2_sell_prc_2']
        dat = ['prc1_month_0', 'prc1_month_1', 'prc1_month_2', 'prc1_month_3', 'prc2_month_0', 'prc2_month_1', 'prc2_month_2', 'prc2_month_3']
        new_names = dict(zip(dis, dat))
        df.rename(columns=new_names, inplace=True)
        
        # get rid of decimals not necessary
        df[dat] = df[dat].astype(np.float32).apply(lambda x: round(x, 3))
        
        return df

# set file path
path = 'C:\\Users\\pmwash\\Desktop\\Re-Engineered Reports\\Automation\\Retail Pricing\\Data\\'

# instantiate frontline query
frontline = AS400FrontLineReport(path)

# cache data in DATASETS dictionary for later access
DATASETS['frontline'] = frontline.data
print('Sample row from Frontline Report\n\n')
print(DATASETS['frontline'].loc[DATASETS['frontline']['product_id']==14012])

## Create `ProductPricing` Class

The `ProductPricing` class is the most important class in this process' architecture.  Other classes, such as `MajorBrandsCompany` are created/instantiated prior to implementing the `ProductPricing` class in order to enable extensability of the process to incorporate unforeseen asks with relative ease.  

In [ ]:
# instantiate classes not yet fleshed out

class MajorBrandsCompany:
    def __init__(self, company_id):
        self.company_id = company_id
        
        
class Director:
    def __init__(self, director_id, director_name):
        self.director_id = director_id
        self.director_name = director_name
        
        
class Manager:
    def __init__(self, manager_id):
        self.manager_id = manager_id
        
        
class Division:
    def __init__(self, division_id):
        self.division_id = division_id
        
        
class ReportingGroup:
    def __init__(self, rpt_group_id):
        self.rpt_group_id = rpt_group_id
        
        
class Supplier:
    def __init__(self, supplier_id, supplier_name):
        self.supplier_id = supplier_id
        self.supplier_name = supplier_name
        
        
class ProductPricing(Product):
    def __init__(self, effective_date, Product, smallest_qty, retail_sell_info, base_prices, 
                 supplier_list_prices, ReportingGroup, available_cases, on_order, 
                 MajorBrandsCompany, Division, priv_label, season_info, cont_type,
                 sell_price_1, sell_price_2, price_group_code, vap, discounts, wine_group_info):
        '''
        ProductPricing extends the Product class by making a new object which inherits all of the 
        characteristics of Product and in addition has information about the next three months of 
        pricing for the Product in question.  
        
        All information comes from the AS400 Frontline Pricing Report.
        
        Instantiate with:
        --------------------------------------------------
         effective_date: date report was run
         Product: base Product class
         smallest_qty: for retail sale
         retail_sell_info: dict of ['rtl_sell_unit', 'rtl_sell_pack'] 
         base_prices: dict of ['base_btl_price', 'base_case_price']
         supplier_list_prices: dict of ['sup_list_case', 'sup_list_btl'] 
         ReportingGroup: object
         available_cases: from frontline report
         on_order: from frontline report
         MajorBrandsCompany: object
         Division: object
         priv_label: boolean
         season_info: dict of season info
         cont_type: container type string
         sell_price_1: dict of ['sell_prc_1', 'nm_sell_prc_1', 'nm_+_1_sell_prc_1', 'nm_+_2_sell_prc_1']
         sell_price_2: dict of ['sell_prc_2', 'nm_sell_prc_2', 'nm_+_1_sell_prc_2', 'nm_+_2_sell_prc_2']
         price_group_code: price group code 
         vap: vap (unsure on semantics)
         discounts: dict of ['disco_kc', 'disco_stl']
         wine_group_info: dict of ['wine_desc', 'wine_comment', 'wine_rpt_grp_type', 'wine_rpt_grp_title']
        '''
        self.effective_date = effective_date
        self.product = Product
        self.smallest_qty = smallest_qty
        self.retail_sell_info = retail_sell_info
        self.base_prices = base_prices
        self.supplier_list_prices = supplier_list_prices
        self.reporting_group = ReportingGroup
        self.available_cases = available_cases
        self.on_order = on_order
        self.company = MajorBrandsCompany
        self.division = Division
        self.priv_label = priv_label
        self.season_info = season_info
        self.cont_type = cont_type
        self.sell_price_1 = sell_price_1
        self.sell_price_2 = sell_price_2
        self.price_group_code = price_group_code
        self.vap = vap
        self.discounts = discounts
        self.wine_group_info = wine_group_info
        
    def __str__(self):
        return '''
        ProductPricing object:
        ----------------------------------------------------------------
        Product:                    %s  -  %s
        Container Type:             %s
        Available Cases:            %s
        Discounts:                  %s
        Wine Grp Info:              %s
        Sell Price 1:               %s
        Sell Price 2:               %s
        Sell Codes:                 %s
        ''' %(self.product.product_id, self.product.description,
             self.cont_type,
             self.available_cases,
             self.discounts,
             self.wine_group_info,
             self.sell_price_1, 
             self.sell_price_2,
             self.product.co_sell_codes)

## Instantiate `ProductPricing` Objects and Save to `PRODUCT_PRICING_MASTER` `dict` 

If a `Product` has a record in the Frontline report then it will be associated with a `ProductPricing` object which extends the base `Product` class.  This extension allows for all pricing information in the Frontline report to be reflected in an enhanced version of a `Product`.  

In [ ]:
# cache columns for all composite attributes of ProductPricing objects
frontline_upc_cols = ['upc_inner', 'upc_inner_type', 'upc_case', 'upc_case_type', 'upc_b_short', 'upc_i_short', 
                      'upc_c_short', 'upc_btl', 'upc_btl_type']
wine_grp_cols = ['wine_desc', 'wine_comment', 'wine_rpt_grp_type', 'wine_rpt_grp_title']
sell_price_1_cols =['prc1_month_0', 'prc1_month_1', 'prc1_month_2', 'prc1_month_3']
sell_price_2_cols = ['prc2_month_0', 'prc2_month_1', 'prc2_month_2', 'prc2_month_3']
season_info_cols = ['seas_code', 'seas_desc', 'seas_start']
base_price_cols = ['base_btl_price', 'base_case_price']
retail_sell_info_cols = ['rtl_sell_unit', 'rtl_sell_pack']
discount_cols = ['disco_kc', 'disco_stl']
supplier_list_cols = ['sup_list_case', 'sup_list_btl'] 

# create master dictionaries to avoid redundant memory allocation
PRODUCT_PRICING_MASTER, REPORTING_GROUP_MASTER, MB_COMPANY_MASTER, DIVISION_MASTER = {}, {}, {}, {}

df = DATASETS['frontline'].copy()
for i, pid in enumerate(df.product_id):

    # update upc list on Product objects before instantiating ProductPricing objects
    upc_frontline_pid = df.loc[i, frontline_upc_cols].to_dict()
    PRODUCT_MASTER[pid].append_upcs(upc_frontline_pid)

    # save column info from central dataset for instantiation
    wine_group_info = df.loc[i, wine_grp_cols].to_dict()
    sell_price_1 = df.loc[i, sell_price_1_cols].to_dict()
    sell_price_2 = df.loc[i, sell_price_2_cols].to_dict()
    season_info = df.loc[i, season_info_cols].to_dict()
    base_prices = df.loc[i, base_price_cols].to_dict()
    retail_sell_info = df.loc[i, retail_sell_info_cols].to_dict()
    discounts = df.loc[i, discount_cols].to_dict()
    supplier_list_prices = df.loc[i, supplier_list_cols].to_dict()

    # if they haven't been yet, instantiate satellite objects ReportingGroup, MajorBrandsCompany & Division
    # these objects currently have limited functionality, this enables extensability later
    if df.loc[i, 'rpt_group'] not in REPORTING_GROUP_MASTER.keys():
        reporting_group_obj = ReportingGroup(df.loc[i, 'rpt_group'])
        REPORTING_GROUP_MASTER[df.loc[i, 'rpt_group']] = reporting_group_obj

    if df.loc[i, 'company'] not in MB_COMPANY_MASTER.keys():
        mb_company_group_obj = MajorBrandsCompany(df.loc[i, 'company'])
        MB_COMPANY_MASTER[df.loc[i, 'company']] = mb_company_group_obj

    if df.loc[i, 'division'] not in DIVISION_MASTER.keys():
        division_obj = Division(df.loc[i, 'division'])
        DIVISION_MASTER[df.loc[i, 'division']] = division_obj

    ##FINAL STEP
    # instantiate ProductPricing objects
    product_pricing_obj = ProductPricing(df.loc[i, 'effective_date'],
                                        PRODUCT_MASTER[pid], 
                                        df.loc[i, 'smallest_qty'],
                                        retail_sell_info,
                                        base_prices,
                                        supplier_list_prices,
                                        reporting_group_obj,
                                        df.loc[i, 'available_cases'],
                                        df.loc[i, 'on_order'],
                                        MB_COMPANY_MASTER[df.loc[i, 'company']],
                                        DIVISION_MASTER[df.loc[i, 'division']],
                                        df.loc[i, 'priv_label'],
                                        season_info,
                                        df.loc[i, 'cont_type'],
                                        sell_price_1,
                                        sell_price_2,
                                        df.loc[i, 'price_grp_code'],
                                        df.loc[i, 'vap_y/n'],
                                        discounts,
                                        wine_group_info)
    
    # cache by product id
    PRODUCT_PRICING_MASTER[pid] = product_pricing_obj
    
del df



# demonstrate object example
print(PRODUCT_PRICING_MASTER[14012])

# Specify `ChainStore` & `RetailCustomer` Classes

`RetailCustomer`s are instantiated and associated with `ChainStore`'s.  Memory usage is optimized by sharing data objects (and thus not duplicated information).

In [ ]:
def fetch_customer_product_dive(path):
    '''
    Fetches Diver Marker for customer-product purchases.  Data source
    must be updated prior to running if new information is desired. 
    '''
    fetchpath = path + 'RETAIL PRICING -- Customer Dive.csv'
    customer_product_dive_df = pd.read_csv(fetchpath, encoding='utf8')
    
    # replace blank space in column names
    customer_product_dive_df.columns = [str(col).replace(' ', '_') for col in customer_product_dive_df.columns]
    customer_product_dive_df.columns = [str(col).lower() for col in customer_product_dive_df.columns]
    
    # get on-premise True or False & fill na in chain_name column with ''
    customer_product_dive_df['on_premise'] = customer_product_dive_df['premise_type'] != 'Off-Premise'
    customer_product_dive_df['chain_name'] = customer_product_dive_df['chain_name'].fillna('')
    
    return customer_product_dive_df


class ChainStore(object):
    def __init__(self, chain_id, chain_name):
        '''
        ChainStore is a simple object that represents a chain store which is an umbrella 
        over its affiliates.  An object is used so functionality may be extended in future.
        
        Instantiate with:
        --------------------------------------------------
         chain_id: unique ChainID from diver
         chain_name: name of chain store
        '''
        self.chain_id = chain_id
        self.chain_name = chain_name
        self.retail_customers = []
        
    def __str__(self):
        return '''
        ChainStore object:
        ----------------------------------------------------------------
        Chain Store:       %s  -  %s
        ''' %(self.chain_id, self.chain_name)
    
    def append_customer(self, RetailCustomer):
        self.retail_customers.append(RetailCustomer)
        
        
class RetailCustomer(ChainStore):
    def __init__(self, customer_id, customer_name, ChainStore, MajorBrandsCompany, warehouse, output_preferences,
                frequency_preferences, on_premise, product_sales):
        '''
        RetailCustomer is a central object to this process.  This object contains attributes, functions
        and affiliations that are helpful in achieving automation of this function. 
        
        Instantiate with:
        --------------------------------------------------
         customer_id: unique customer identifier
         customer_name: name of customer location
         ChainStore: ChainStore object (max of 1)
         MajorBrandsCompany: MajorBrandsCompany object (max of 1)
         output_preferences: TBD
         frequency_preferences: TBD
         on_premise: True if on-premise, o.w. False
         product_sales: data for std cases and dollars sold by product for RetailCustomer ytd
        '''
        self.customer_id = customer_id
        self.customer_name = customer_name
        self.chain_store = ChainStore
        self.company = MajorBrandsCompany
        self.warehouse = warehouse
        self.output_preferences = output_preferences
        self.frequency_preferences = frequency_preferences
        self.on_premise = on_premise
        self.product_sales = product_sales
        self.products_carried = []
        self.products_carried_pricing = []
        self.number_products_carried = len(self.products_carried)
        
    def __str__(self):
        return '''
        RetailCustomer object:
        ----------------------------------------------------------------
        Customer:           %s  -  %s
        Warehouse:          %s
        On Premise:         %s
        MB Products:        %s
        
        Chain Affiliate:    %s
        ''' %(self.customer_id, self.customer_name, self.warehouse, self.on_premise, 
              self.number_products_carried, self.chain_store)
    
    def append_product(self, Product):
        self.products_carried.append(Product)
        self.number_products_carried = len(self.products_carried)
        
    def append_product_pricing(self, ProductPricing):
        self.products_carried_pricing.append(ProductPricing)
        
    def list_products(self):
        '''
        Basic list of all products carried by the RetailCustomer in question.
        '''
        prod_ids = [p.product_id for p in self.products_carried] 
        prod_desc = [p.description for p in self.products_carried]
        prod_size = [p.size_code for p in self.products_carried]
        dat = {'product_id': prod_ids, 'description': prod_desc, 'size_code': prod_size}
        prd_df = pd.DataFrame(dat).set_index('product_id')
        prd_df = prd_df.join(self.product_sales).sort_values('dollars', ascending=False)
        
        return prd_df
        
    def list_product_prices(self):
        '''
        Lists all products carried by the RetailCustomer in question as well as their prc1 and 
        prc2 values for the next three months.
        '''
        prod_ids = [p.product.product_id for p in self.products_carried_pricing]
        prod_desc = [p.product.description for p in self.products_carried_pricing]
        
        # define prc1 and prc2 dataframes then merge
        prc1 = [p.sell_price_1 for p in self.products_carried_pricing]
        prc2 = [p.sell_price_2 for p in self.products_carried_pricing]
        prc1_df = pd.DataFrame(prc1, index=prod_ids)
        prc2_df = pd.DataFrame(prc2, index=prod_ids)
        prc_df = prc1_df.join(prc2_df)
        
        # next three months names
        get_pricing_month = lambda i: format(dt.strptime(str(dt.now().month + int(i)), '%m'), '%b').lower()
        cols_now = [str(col).split('month_') for col in prc_df.columns]
        next3_mo = [str(col[0]) + str(get_pricing_month(col[-1])) for col in cols_now]
        
        # rename
        prc_df.rename(columns=dict(zip(prc_df.columns.tolist(), next3_mo)), inplace=True)
        
        # merge with basic product list
        prc_df = self.list_products().join(prc_df, how='outer')
        prc_df = prc_df.join(self.list_productpricing_attributes(), how='outer')
        prc_df.index.name = 'product_id'
        
        return prc_df
    
    def list_productpricing_attributes(self):
        '''
        Lists all attributes available to the ProductPricing object.
        '''
        productpricing_data = []
        for i, prd_prc in enumerate(self.products_carried_pricing):
            dat = {'product_id': prd_prc.product.product_id,
                  'cont_type': prd_prc.cont_type,
                  'base_btl_price': prd_prc.base_prices['base_btl_price'],
                  'base_case_price': prd_prc.base_prices['base_case_price'],
                  'company': prd_prc.company.company_id,
                  'available_cases': prd_prc.available_cases,
                  'on_order': prd_prc.on_order,
                  'discount_kc': prd_prc.discounts['disco_kc'],
                  'discount_stl': prd_prc.discounts['disco_stl'],
                  'effective_date': prd_prc.effective_date,
                  'price_group_code': prd_prc.price_group_code,
                  'priv_label': prd_prc.priv_label,
                  'rtl_sell_pack': prd_prc.retail_sell_info['rtl_sell_pack'],
                  'rtl_sell_unit': prd_prc.retail_sell_info['rtl_sell_unit'],
                  'seas_code': prd_prc.season_info['seas_code'],
                  'seas_desc': prd_prc.season_info['seas_desc'],
                  'seas_start': prd_prc.season_info['seas_start'],
                  'smallest_qty': prd_prc.smallest_qty,
                  'sup_list_btl': prd_prc.supplier_list_prices['sup_list_btl'],
                  'sup_list_case': prd_prc.supplier_list_prices['sup_list_case'],
                  'vap': prd_prc.vap,
                  'wine_comment': prd_prc.wine_group_info['wine_comment'],
                  'wine_desc': prd_prc.wine_group_info['wine_desc'],
                  'wine_rpt_grp_title': prd_prc.wine_group_info['wine_rpt_grp_title'],
                  'wine_rpt_grp_type': prd_prc.wine_group_info['wine_rpt_grp_type']}
            productpricing_data.append(dat)
        
        productpricing_df = pd.DataFrame(productpricing_data).set_index('product_id')
        productpricing_df.index.name = 'product_id'
        
        return productpricing_df

### Instantiate `ChainStore` & `RetailCustomer` Class Objects

In [ ]:
# read in Diver marker that contains chain/customer and products carried data
tmp_path = 'C:\\Users\\pmwash\\Desktop\\Re-Engineered Reports\\Generalized Lookup Data\\'
custprod_dive_df = fetch_customer_product_dive(tmp_path)


# accumulate all chain stores into a dictionary
CHAIN_STORE_MASTER = {}
for ix, row in custprod_dive_df.loc[custprod_dive_df.chainid != 0, ['chainid', 'chain_name']].drop_duplicates().iterrows():
    CHAIN_STORE_MASTER[row['chainid']] = ChainStore(row['chainid'], row['chain_name'])
    
    
# accumulate all customer objects into a dictionary
RETAIL_CUSTOMER_MASTER = {}
iter_cols = ['customer_id', 'customer', 'on_premise', 'chainid', 'warehouse']
for ix, row in custprod_dive_df[iter_cols].drop_duplicates().iterrows():
    
    if row['chainid'] != 0: this_chain = CHAIN_STORE_MASTER[row['chainid']]
    else: this_chain = None
        
    # get sales data by product for customer in iteration
    this_customers_product_sales = custprod_dive_df.loc[custprod_dive_df.customer_id == row['customer_id'], 
                                                        ['product_id', 'dollars', 'std_cases']].set_index('product_id')
    
    # instantiate customer object
    customer_obj = RetailCustomer(customer_id=row['customer_id'], 
                                  customer_name=row['customer'], 
                                  ChainStore=this_chain, 
                                  MajorBrandsCompany=None,
                                  warehouse=row['warehouse'],
                                  output_preferences={},
                                  frequency_preferences={}, 
                                  on_premise=row['on_premise'],
                                  product_sales=this_customers_product_sales)
    
    # add products to customer object
    is_this_customer = custprod_dive_df.customer_id == row['customer_id'] #all rows where thiscust
    this_customers_products = custprod_dive_df.loc[is_this_customer, 'product_id'].tolist()
    for pid in this_customers_products:
        try: 
            # append cust obj with product and product pricing
            customer_obj.append_product(PRODUCT_MASTER[pid])
            customer_obj.append_product_pricing(PRODUCT_PRICING_MASTER[pid])
        except KeyError: 
            pass
    
    # save finished object to dictionary for access and reference later
    RETAIL_CUSTOMER_MASTER[row['customer_id']] = customer_obj


# test to make sure it works as planned
print(RETAIL_CUSTOMER_MASTER[13610])

---------------------------------------------------------------------------------------------







---------------------------------------------------------------------------------------------
**END DEVELOPMENT**

---------------------------------------------------------------------------------------------












# Demonstration of the Process

### Here we have a `RetailCustomer` object for QT

In [ ]:
print(RETAIL_CUSTOMER_MASTER[13610])

### Who has purchased certain products from us

In [ ]:
RETAIL_CUSTOMER_MASTER[13610].list_products().head()

### Each `Product` they bought has a `ProductPricing` object if it is an active item

#### Let's review one of this `RetailCustomer`'s `Product`'s `ProductPricing` objects

`Product` is the base class for `ProductPricing` and thus inherits some of its functionality.  Note the difference between the two objects -- `Product` is extended by `ProductPricing` and cannot exist without one.  

In [ ]:
print(PRODUCT_MASTER[4291567])
print(PRODUCT_PRICING_MASTER[4291567])

### Now the payload: what are the next three months of `ProductPrices` for this `RetailCustomer`'s `Product`'s?

A summary of all `ProductPricing` for each `RetailCustomer` can be accessed through the `RetailCustomer` object's class method called `list_product_prices()`.  This function will return all of the products associated with that `RetailCustomer` and all the details pertinent to pricing.  This function deliberately returns more data than is likely to be needed since it is easier to prune the data than to grow it.

By running this function for all retail customers who desire near-ter pricing we can get the majority of the information distributed rapidly.  

In [ ]:
print(RETAIL_CUSTOMER_MASTER[13610].list_product_prices().head())

### Object-oriented design enables us to go deep and broad in a highly efficient & non-redundant way

The process above gets us 90% of the way there for many of our `RetailCustomer`'s needs.  In some cases it gets us all the way there.  

At its essence this process is to be used when a `RetailCustomer` wants the next 3 months of `ProductPricing` for `Product`s of ours they sell.  By using object-oriented design we are able to achieve much of the heavy lifting by centralizing the data processing (versus satellites doing it in Excel).  This enables extensions to the process that can be customized to adhere to customer needs and our capabilities.  

# Future Work

- How is beer different?  Are there more data sources for beer?
- Is what is done so far capable of being extended for all scenarios?  Which can it not meet?
- How can we use `.xlsx` files to keep track of customer preferences (i.e. which to include, special columns, which UPC, etc.)
- Do customers all have realistic expectations of what we can do (e.g. no more than 3 months out)?
- Which processes still rely on Excel?  How can we eliminate the need for Excel?